In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
base_url = os.getenv("AZURE_OPENAI_ENDPOINT")
# The API key for your Azure OpenAI resource.
api_key = os.getenv("AZURE_OPENAI_API_KEY")
# Setting up the deployment name
gpt_model_name = os.getenv("AZURE_OPENAI_GPT_MODEL")
embedding_model_name = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL")
# Currently Chat Completions API have the following versions available: 2023-03-15-preview
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
api_type = "azure"

In [ ]:
from phoenix.evals import (
    RAG_RELEVANCY_PROMPT_TEMPLATE,
    RAG_RELEVANCY_PROMPT_RAILS_MAP,
    OpenAIModel,
    download_benchmark_dataset,
    llm_classify,
)
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, ConfusionMatrixDisplay

# Download the benchmark golden dataset
df = download_benchmark_dataset(
    task="binary-relevance-classification", dataset_name="wiki_qa-train"
)
# Sample and re-name the columns to match the template
df = df.sample(100)
df = df.rename(
    columns={
        "query_text": "input",
        "document_text": "reference",
    },
)
model = OpenAIModel(
    model=gpt_model_name,
    api_key=api_key,
    azure_endpoint=base_url,
    api_version=api_version,
)
rails =list(RAG_RELEVANCY_PROMPT_RAILS_MAP.values())
df[["eval_relevance"]] = llm_classify(df, model, RAG_RELEVANCY_PROMPT_TEMPLATE, rails)
#Golden dataset has True/False map to -> "irrelevant" / "relevant"
#we can then scikit compare to output of template - same format
y_true = df["relevant"].map({True: "relevant", False: "irrelevant"})
y_pred = df["eval_relevance"]

# Compute Per-Class Precision, Recall, F1 Score, Support
precision, recall, f1, support = precision_recall_fscore_support(y_true, y_pred)